In [1]:
from collections import Counter, defaultdict, namedtuple, deque
from itertools   import permutations, combinations, product, chain
from functools   import lru_cache
from typing      import Dict, Tuple, Set, List, Iterator, Optional, Union, Iterable, Sequence
from dataclasses import dataclass


import heapq
import operator
import math
import ast
import sys
import re
import statistics

import numpy as np

Initial methods and `do`/`data` are borrowed from Peter Norvig's Advent of Code solutions

In [2]:
def data(day: int, parser=str, sep='\n') -> list:
    "Split the day's input file into sections separated by `sep`, and apply `parser` to each."
    sections = open(f'data/input{day}.txt').read().rstrip().split(sep)
    return [parser(section) for section in sections]
     
def do(day, *answers) -> Dict[int, int]:
    "E.g., do(3) returns {1: day3_1(in3), 2: day3_2(in3)}. Verifies `answers` if given."
    g = globals()
    got = []
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](g[f'in{day}']))
            if len(answers) >= part: 
                assert got[-1] == answers[part - 1], (
                    f'{fname}(in{day}) got {got[-1]}; expected {answers[part - 1]}')
    return got

In [3]:
def quantify(iterable, pred=bool) -> int:
    """Count the number of items in iterable for which pred is true."""
    return sum(1 for item in iterable if pred(item))

def first(iterable, default=None) -> object:
    """Return first item in iterable, or default."""
    return next(iter(iterable), default)

def rest(sequence) -> object: 
    return sequence[1:]

def multimap(items: Iterable[Tuple]) -> dict:
    "Given (key, val) pairs, return {key: [val, ....], ...}."
    result = defaultdict(list)
    for (key, val) in items:
        result[key].append(val)
    return result

def ints(text: str) -> Tuple[int]:
    """Return a tuple of all the integers in text."""
    return tuple(map(int, re.findall('-?[0-9]+', text)))

def atoms(text: str, ignore=r'', sep=None) -> Tuple[Union[int, str]]:
    """Parse text into atoms (numbers or strs), possibly ignoring a regex."""
    if ignore:
        text = re.sub(ignore, '', text)
    return tuple(map(atom, text.split(sep)))

def atom(text: str) -> Union[float, int, str]:
    """Parse text into a single float or int or str."""
    try:
        val = float(text)
        return round(val) if round(val) == val else val
    except ValueError:
        return text
    
def dotproduct(A, B) -> float: 
    return sum(a * b for a, b in zip(A, B))

def mapt(fn, *args):
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

cat = ''.join
flatten = chain.from_iterable
Char = str # Type used to indicate a single character

# Day 1: Sonar Sweep

In [4]:
in1: List[int] = data(1, int)

In [5]:
def day1_1(nums: List[int]) -> int:
    return sum([1 for i in range(len(nums) - 1) if nums[i+1] > nums[i]])

def day1_2(nums: List[int]) -> int:
    return day1_1([sum(nums[i:i+3]) for i in range(len(nums) - 2)])

In [6]:
do(1, 1521, 1543)

[1521, 1543]

# Day 2: Dive!

In [7]:
Command = Tuple[str, int]

def parse_command(line: str) -> Command:
    direction, quantity = line.split(' ')
    return (direction, int(quantity))

in2: List[Command] = data(2, parse_command)


In [8]:
def day2_1(commands: List[Command]) -> int:
    pos = [0, 0]
    for command in commands:
        direction = command[0]
        quantity = command[1]
        if direction == "forward":
            pos[0] += quantity
        if direction == "up":
            pos[1] -= quantity
        if direction == "down":
            pos[1] += quantity
    return math.prod(pos)

def day2_2(commands: List[Command]) -> int:
    pos = [0, 0]
    aim = 0
    for command in commands:
        direction = command[0]
        quantity = command[1]
        if direction == "forward":
            pos[0] += quantity
            pos[1] += quantity * aim
        if direction == "up":
            aim -= quantity
        if direction == "down":
            aim += quantity
    return math.prod(pos)

In [9]:
do(2, 1383564, 1488311643)

[1383564, 1488311643]

# Day 3: Binary Diagnostic

In [10]:
BinaryReportLine = List[int]

def parse_binary_report_line(line: str) -> BinaryReportLine:
    return list(map(int, list(line)))

in3: List[BinaryReportLine] = data(3, parse_binary_report_line)

In [11]:
def day3_1(report_lines: List[BinaryReportLine]) -> int:
    inverted = list(map(list, zip(*report_lines)))
    gamma = "".join(["1" if sum(col) > len(col)/2 else "0" for col in inverted])
    epsilon = "".join(["1" if v == "0" else "0" for v in gamma])
    return int(gamma, 2) * int(epsilon, 2)

def filter_report(report_lines: List[BinaryReportLine], filter_value_fn) -> BinaryReportLine:
    filtered = report_lines
    i = 0
    while len(filtered) > 1:
        column_vals = [row[i] for row in filtered]
        total = len(column_vals)
        ones = sum(column_vals)
        filter_value = filter_value_fn(total, ones)
        filtered = [row for row in filtered if row[i] == filter_value]
        i+=1
    return filtered[0]

def day3_2(report_lines: List[BinaryReportLine]) -> int:
    oxygen = filter_report(report_lines, lambda total, ones: 1 if ones >= total/2 else 0)
    co2 = filter_report(report_lines, lambda total, ones: 1 if ones < total/2 else 0)
    return int("".join(map(str, oxygen)), 2) * int("".join(map(str, co2)), 2)



In [12]:
do(3, 2498354, 3277956)

[2498354, 3277956]

# Day 4: Giant Squid

In [13]:
def day4_data() -> Tuple[List[str], List[List[List[str]]]]:
    parsed_data = data(4, sep="\n\n")
    called_numbers = parsed_data[0].split(",")
    boards = []
    for unparsed_board in parsed_data[1:]:
        board = []
        board_lines = unparsed_board.split("\n")
        for line in board_lines:
            board.append(line.split())
        boards.append(board)
    return called_numbers, boards

in4: Tuple[List[str], List[List[List[str]]]] = day4_data()

In [14]:
class Board:
    def __init__(self, board):
        self.board = board
        self.wins = []
        for row in board:
            self.wins.append(set(row))

        self.won = False

        for i in range(len(board[0])):
            col_win = set()
            for row in board:
                col_win.add(row[i])
            self.wins.append(col_win)

    def all_values(self) -> List[str]:
        return list(set([item for row in self.board for item in row]))

    def get_score(self) -> int:
        uncalled_values = set()
        for win in self.wins:
            uncalled_values |= win
        return sum(map(int, uncalled_values))


    
# Return a map from the numbers in the boards to which board they are in
# For eact board, build a set of all the possible winning paths
def build_bingo_map(boards: List[List[List[str]]]) -> Dict[str, List[Board]]:
    m = defaultdict(list)
    for board in boards:
        b = Board(board)
        for v in b.all_values():
            m[v].append(b)
    return m


def day4_1(data: Tuple[List[str], List[List[List[str]]]]) -> int:
    calls, boards = data
    bingo_map = build_bingo_map(boards)
    for call in calls:
        boards_with_number = bingo_map[call]
        for board in boards_with_number:
            won = False
            for win in board.wins:
                if call in win:
                    win.remove(call)
                    if len(win) == 0:
                        won = True
            if won:
                return board.get_score() * int(call)

    return 0

def day4_2(data: Tuple[List[str], List[List[List[str]]]]) -> int:
    winning_boards = []
    calls, boards = data
    bingo_map = build_bingo_map(boards)
    for call in calls:
        boards_with_number = bingo_map[call]
        for board in boards_with_number:
            won = False
            for win in board.wins:
                if call in win:
                    win.remove(call)
                    if len(win) == 0:
                        won = True
            if won and not board.won:
                winning_boards.append((board, int(call)))
                board.won = True
        if len(winning_boards) == len(boards):
            return winning_boards[-1][0].get_score() * winning_boards[-1][1]

    return 0


do(4, 41668, 10478)    


[41668, 10478]

# Day 5: Hydrothermal Venture

In [15]:
VentLine = Tuple[Tuple[int, int], Tuple[int, int]]

def parse_day5(line: str) -> VentLine:
    return tuple(mapt(int, v.strip().split(",")) for v in line.split("->"))

in5: List[VentLine] = data(5, parse_day5)

In [16]:
def day5_1(vent_lines: List[VentLine]) -> int:
    vents = defaultdict(int)
    for line in vent_lines:
        if line[0][0] == line[1][0]:
            a = line[0][1]
            b = line[1][1]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(line[0][0], i)] += 1
        if line[0][1] == line[1][1]:
            a = line[0][0]
            b = line[1][0]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(i, line[0][1])] += 1
        
    return sum([1 for count in vents.values() if count >= 2])

def get_step(start: int, end: int) -> int:
    if end > start:
        return 1
    if start > end:
        return -1
    return 0

def day5_2(vent_lines: List[VentLine]) -> int:
    vents = defaultdict(int)
    for line in vent_lines:
        if line[0][0] == line[1][0]:
            a = line[0][1]
            b = line[1][1]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(line[0][0], i)] += 1
        elif line[0][1] == line[1][1]:
            a = line[0][0]
            b = line[1][0]
            start = min(a, b)
            end = max(a, b)
            for i in range(start, end + 1):
                vents[(i, line[0][1])] += 1
        else:
            # Diagonal
            xstep = get_step(line[0][0], line[1][0])
            ystep = get_step(line[0][1], line[1][1])
            for (x, y) in zip(range(line[0][0], line[1][0] + xstep, xstep), range(line[0][1], line[1][1] + ystep, ystep)):
                vents[(x, y)] += 1
        
    return sum([1 for count in vents.values() if count >= 2])

do(5, 4728, 17717)


[4728, 17717]

# Day 6: Lanternfish

In [17]:
def parse_day5(line: str) -> VentLine:
    return tuple(mapt(int, v.strip().split(",")) for v in line.split("->"))

in6: List[int] = first(data(6, lambda l: list(map(int, l.split(",")))))

In [18]:
def lanternfish_after_n_days(timers: List[int], days: int) -> int:
    m = defaultdict(int)
    for timer in timers:
        m[timer] += 1

    for i in range(days):
        next_day = defaultdict(int)
        for t in m:
            if t == 0:
                next_day[8] += m[t]
                next_day[6] += m[t]
            else:
                next_day[t-1] += m[t]
        m = next_day
    return sum(count for count in m.values())

def day6_1(timers: List[int]) -> int:
    return lanternfish_after_n_days(timers, 80)

def day6_2(timers: List[int]) -> int:
    return lanternfish_after_n_days(timers, 256)

do(6, 386640, 1733403626279)

[386640, 1733403626279]

# Day 7: The Treachery of Whales

In [19]:
in7: List[int] = data(7, parser=int, sep=",")

In [20]:
def day7_1(locations: List[int]) -> int:
    median = int(statistics.median(in7))
    return sum([abs(median - v) for v in locations])

def day7_2(locations: List[int]) -> int:
    left = min(locations)
    right = max(locations)
    possible_destinations = list(range(left, right+1))
    fuel_for_destination = {}
    for destination in possible_destinations:
        destination_fuel = 0
        for location in locations:
            distance = abs(location - destination)
            destination_fuel += int((distance * (distance + 1)) / 2)
        fuel_for_destination[destination] = destination_fuel
    return min(fuel_for_destination.values())

do(7, 348664, 100220525)

[348664, 100220525]

# Day 8: Seven Segment Search

In [21]:
SignalPatterns = List[str]
DigitOutputValues = Tuple[str, str, str]

def parse_day8(line: str) -> Tuple[SignalPatterns, DigitOutputValues]:
    patterns, outputs = line.split("|")
    return patterns.strip().split(), tuple(outputs.strip().split())

in8: List[Tuple[SignalPatterns, DigitOutputValues]] = data(8, parser=parse_day8)

In [22]:
def day8_1(display_lines: List[Tuple[SignalPatterns, DigitOutputValues]]) -> int:
    digit_count = 0
    unique_digit_lengths = frozenset([2, 4, 3, 7])
    for _, output_values in display_lines:
        digit_count += sum([1 for value in output_values if len(value) in unique_digit_lengths])
    return digit_count

def day8_2(display_lines: List[Tuple[SignalPatterns, DigitOutputValues]]) -> int:
    numbers = mapt(frozenset, ["abcefg", "cf", "acdeg", "acdfg", "bcdf", "abdfg", "abdefg", "acf", "abcdefg", "abcdfg"])

    length_number_mappings = multimap([(len(encoding), number) for number, encoding in enumerate(numbers)])

    total = 0

    for signal_patterns, output_values in display_lines:
        # Mapping from number to the signal set
        correct_letter_mappings = {}
        length_pattern_mappings = defaultdict(set)
        for signal_pattern in signal_patterns:
            length_pattern_mappings[len(signal_pattern)].add(frozenset(signal_pattern))

        # Only known length values
        for length, patterns in length_pattern_mappings.items():
            numbers_of_length = length_number_mappings[length]
            if len(numbers_of_length) == 1:
                correct_letter_mappings[numbers_of_length[0]] = frozenset(list(patterns)[0])

        # Find 2
        for pattern in list(length_pattern_mappings[5]):
            if len(pattern - correct_letter_mappings[4]) == 3:
                correct_letter_mappings[2] = pattern
                length_pattern_mappings[5].remove(pattern)

        # Find 3, 5
        for pattern in list(length_pattern_mappings[5]):
            if len(pattern - correct_letter_mappings[2]) == 1:
                correct_letter_mappings[3] = pattern
                length_pattern_mappings[5].remove(pattern)
        assert len(length_pattern_mappings[5]) == 1
        correct_letter_mappings[5] = length_pattern_mappings[5].pop()

        # Find 0
        for pattern in list(length_pattern_mappings[6]):
            if len(pattern - correct_letter_mappings[5]) == 2:
                correct_letter_mappings[0] = pattern
                length_pattern_mappings[6].remove(pattern)

        # Find 6, 9
        for pattern in list(length_pattern_mappings[6]):
            if len(pattern - correct_letter_mappings[1]) == 5:
                correct_letter_mappings[6] = pattern
                length_pattern_mappings[6].remove(pattern)
        assert len(length_pattern_mappings[6]) == 1
        correct_letter_mappings[9] = length_pattern_mappings[6].pop()
        assert len(correct_letter_mappings) == 10

        number_lookup = {pattern: number for number, pattern in correct_letter_mappings.items()}
        output_patterns = mapt(frozenset, output_values)
        output_number = (number_lookup[output_patterns[0]] * 1000) + (number_lookup[output_patterns[1]] * 100) + (number_lookup[output_patterns[2]] * 10) + number_lookup[output_patterns[3]]
        total += output_number        
        
    return total

do(8, 412, 978171)

[412, 978171]

# Day 9: Smoke Basin

In [23]:
def parse_day9(line: str) -> List[int]:
    return list(map(int, list(line)))
in9: List[List[int]] = data(9, parser=parse_day9)

In [24]:
def day9_find_low_points(grid: List[List[int]]) -> List[Tuple[int, int]]:
    low_points = []
    for i in range(len(grid)):
        for j in range(len(grid[i])):
            cell_value = grid[i][j]
            is_low_point = all(cell_value < grid[n_i][n_j] for n_i, n_j in [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)] if n_i >= 0 and n_i < len(grid) and n_j >= 0 and n_j < len(grid[n_i]))
            if is_low_point:
                low_points.append((i, j))
    return low_points

def day9_find_basin_size(low_point: Tuple[int, int], grid: List[List[int]]) -> int:
    q = deque([low_point])
    seen = set()
    size = 0
    while q:
        i, j = q.popleft()
        if (i, j) in seen:
            continue
        seen.add((i, j))
        if grid[i][j] == 9:
            continue
        
        size += 1
        q.extend([(n_i, n_j) for n_i, n_j in [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)] if n_i >= 0 and n_i < len(grid) and n_j >= 0 and n_j < len(grid[n_i])])
    return size


def day9_1(grid: List[List[int]]) -> int:
    return sum(grid[i][j] + 1 for i, j in day9_find_low_points(grid))

def day9_2(grid: List[List[int]]) -> int:
    low_points = day9_find_low_points(grid)
    basin_sizes = sorted([day9_find_basin_size(low_point, grid) for low_point in low_points], reverse=True)
    return math.prod(basin_sizes[:3])



do(9, 560, 959136)

[560, 959136]

# Day 10: Syntax Scoring

In [25]:
in10: List[str] = data(10)

In [26]:
def day10_get_corrupted_line_score(line: str) -> int:
    scores = {
        ')': 3,
        ']': 57,
        '}': 1197,
        '>': 25137,
    }
    matches = {
        '(': ')',
        '[': ']',
        '{': '}',
        '<': '>',
    }
    stack = []
    for c in line:
        if c in matches:
            stack.append(c)
            continue
        else:
            if len(stack) == 0:
                # Unexpected closing character, but no expected character.
                return scores[c]
            if matches[stack[-1]] == c:
                stack.pop()
            else:
                return scores[c]

    return 0

def day10_get_score_incomplete_lines(line: str) -> int:
    scores = {
        ')': 1,
        ']': 2,
        '}': 3,
        '>': 4,
    }
    matches = {
        '(': ')',
        '[': ']',
        '{': '}',
        '<': '>',
    }
    stack = []
    for c in line:
        if c in matches:
            stack.append(c)
            continue
        else:
            if len(stack) == 0:
                # Unexpected closing character, but no expected character.
                return 0
            if matches[stack[-1]] == c:
                stack.pop()
            else:
                return 0

    #print("Line:", line)
    #print("Stack:", stack)
    score = 0
    while stack:
        score *= 5
        score += scores[matches[stack.pop()]]
    return score

def day10_1(lines: List[str]) -> int:
    return sum(day10_get_corrupted_line_score(line) for line in lines)

def day10_2(lines: List[str]) -> int:
    return int(statistics.median([day10_get_score_incomplete_lines(line) for line in lines if day10_get_corrupted_line_score(line) == 0]))


do(10, 366027, 1118645287)


[366027, 1118645287]

# Day 11: Dumbo Octopus

In [27]:
def parse_day11(line: str) -> List[int]:
    return list(map(int, list(line)))
in11: List[List[int]] = data(11, parser=parse_day11)

In [28]:
def day11_build_neighbors_fn(max_r, max_c):
    def fn(r, c) -> List[Tuple[int, int]]:
        candidates = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        possible_values = [(r + i, c + j) for i, j in candidates]
        return [v for v in possible_values if v[0] >= 0 and v[1] >= 0 and v[0] < max_r and v[1] < max_c]
    return fn

def day11_step(previous_generation: List[List[int]]) -> List[List[int]]:
    neighbors_fn = day11_build_neighbors_fn(len(previous_generation), len(previous_generation[0]))

    # First, the energy level of each octopus increases by 1
    next_generation = [[v + 1 for v in row] for row in previous_generation]
    # Then, any octopus with an energy level greater than 9 flashes. 
    # This increases the energy level of all adjacent octopuses by 1, 
    # including octopuses that are diagonally adjacent. 
    # If this causes an octopus to have an energy level greater than 9, it also flashes. 
    # This process continues as long as new octopuses keep having their energy level increased beyond 9. 
    # (An octopus can only flash at most once per step.)
    flashed = set()
    will_flash = deque()
    for r in range(len(next_generation)):
        for c in range(len(next_generation[r])):
            if next_generation[r][c] > 9:
                will_flash.append((r, c))

    while will_flash:
        oct = will_flash.popleft()
        if oct in flashed:
            continue
        flashed.add(oct)
        r, c = oct
        for neighbor in neighbors_fn(r, c):
            neighbor_r, neighbor_c = neighbor
            next_generation[neighbor_r][neighbor_c] += 1
            if next_generation[neighbor_r][neighbor_c] > 9:
                will_flash.append(neighbor)

    # Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.
    for r in range(len(next_generation)):
        for c in range(len(next_generation[r])):
            if next_generation[r][c] > 9:
                next_generation[r][c] = 0
        
    return next_generation



def day11_1(grid: List[List[int]]) -> int:
    flashes = 0
    current_generation = grid
    for i in range(100):
        current_generation = day11_step(current_generation)
        flashes += quantify(flatten(current_generation), lambda v: v == 0)

    return flashes

def day11_2(grid: List[List[int]]) -> int:
    current_generation = grid
    generation = 0
    while any([v != 0 for v in flatten(current_generation)]):
        current_generation = day11_step(current_generation)
        generation += 1

    return generation

do(11, 1773, 494)

[1773, 494]

# Day 12: Passage Pathing

In [29]:
def parse_day12(line: str) -> List[int]:
    return tuple(line.split('-'))

def build_graph_day12(edges: List[Tuple[str, str]]) -> Dict[str, List[str]]:
    g = defaultdict(list)
    for edge in edges:
        g[edge[0]].append(edge[1])
        g[edge[1]].append(edge[0])
    return g
in12: Dict[str, List[str]] = build_graph_day12(data(12, parser=parse_day12))

In [30]:
def day12_is_small_cave(cave_name: str) -> bool:
    return cave_name.islower()

def day12_is_big_cave(cave_name: str) -> bool:
    return cave_name.isupper()

def day12_calculate_paths(graph: Dict[str, List[str]], visit_limits: Dict[str, int]) -> List[Tuple[str, ...]]:
    paths = []
    start_node = "start"
    end_node = "end"
    visited = defaultdict(int)
    def dfs(path, current_node):
        if current_node in visited and visited[current_node] >= visit_limits[current_node]:
            return
        path.append(current_node)
        # If we are at the end, return and add the path to the list of paths.
        if current_node == end_node:
            paths.append(tuple(path))
            path.pop()
            return            
        # We don't want to visit a small cave twice.
        if current_node in visit_limits:
            visited[current_node]+=1
        
        for neighbor in graph[current_node]:
            dfs(path, neighbor)
        if current_node in visited:
            visited[current_node] -= 1
        path.pop()
    dfs([], start_node)

    return paths    

def day12_1(graph: Dict[str, List[str]]) -> int:
    visit_limits = {cave_name: 1 for cave_name in graph if day12_is_small_cave(cave_name)}
    return len(day12_calculate_paths(graph, visit_limits))

def day12_2(graph: Dict[str, List[str]]) -> int:
    paths = []
    visit_limits = {cave_name: 1 for cave_name in graph if day12_is_small_cave(cave_name)}
    for cave_name in graph:
        if cave_name not in {"start", "end"} and day12_is_small_cave(cave_name):
            visit_limits[cave_name]+=1
            paths.extend(day12_calculate_paths(graph, visit_limits))
            visit_limits[cave_name]-=1
    return len(set(paths))



do(12, 3410, 98796)

[3410, 98796]

# Day 13: Transparent Origami

In [31]:
def parse_day13(chunks: List[str]) -> Tuple[List[Tuple[int, int]], List[Tuple[str, int]]]:
    assert len(chunks) == 2

    points = [mapt(int, line.split(",")) for line in chunks[0].split("\n")]
    folds = []
    for line in chunks[1].split("\n"):
        folds.append(atoms(line, "fold along ", "="))
    return points, folds

in13: Tuple[List[Tuple[int, int]], List[Tuple[str, int]]] =  parse_day13(data(13, sep="\n\n"))

In [32]:
def day13_fold(points: List[Tuple[int, int]], fold: Tuple[str, int]) -> List[Tuple[int, int]]:
    new_points = set()
    for x, y in points:
        if fold[0] == "x":
            assert x != fold[1]
            if x < fold[1]:
                new_points.add((x, y))
            else:
                new_x = fold[1] - (x - fold[1])
                if new_x >= 0:
                    new_points.add((new_x, y))
        if fold[0] == "y":
            assert y != fold[1]
            if y < fold[1]:
                new_points.add((x, y))
            else:
                new_y = fold[1] - (y - fold[1])
                if new_y >= 0:
                    new_points.add((x, new_y))

    return list(new_points)

def day13_print(points: List[Tuple[int, int]]) -> None:
    max_x = max(x for x, _ in points)
    max_y = max(y for _, y in points)
    point_set = set(points)
    for y in range(max_y + 1):
        for x in range(max_x + 1):
            if (x, y) in point_set:
                print("#", end="")
            else:
                print(" ", end="")
        print()


def day13_1(data: Tuple[List[Tuple[int, int]], List[Tuple[str, int]]]) -> int:
    return len(day13_fold(data[0], data[1][0]))


def day13_2(data: Tuple[List[Tuple[int, int]], List[Tuple[str, int]]]) -> str:
    points = data[0]
    for fold in data[1]:
        points = day13_fold(points, fold)
    
    day13_print(points)
    return "BLKJRBAG"

do(13, 755, "BLKJRBAG")

###  #    #  #   ## ###  ###   ##   ## 
#  # #    # #     # #  # #  # #  # #  #
###  #    ##      # #  # ###  #  # #   
#  # #    # #     # ###  #  # #### # ##
#  # #    # #  #  # # #  #  # #  # #  #
###  #### #  #  ##  #  # ###  #  #  ###


[755, 'BLKJRBAG']

# Day 14: Extended Polymerization

In [33]:
def parse_day14(chunks: List[str]) -> Tuple[str, Dict[str, str]]:
    assert len(chunks) == 2

    template = chunks[0]
    insertion_rules = dict([tuple(line.split(" -> ")) for line in chunks[1].split("\n")])
    return template, insertion_rules

in14: Tuple[str, Dict[str, str]] =  parse_day14(data(14, sep="\n\n"))

In [34]:
def day14_step1(template: str, insertion_rules: Dict[str, str]) -> str:
    next_template = ""
    for i in range(len(template) - 1):
        pair = template[i:i+2]
        next_template += template[i]
        next_template += insertion_rules.get(pair, "")
    next_template += template[-1]
    return next_template


def day14_1(data: Tuple[str, Dict[str, str]]) -> int:
    template = data[0]
    insertion_rules = data[1]

    for i in range(10):
        template = day14_step1(template, insertion_rules)

    freq_map = defaultdict(int)
    for c in template:
        freq_map[c] += 1

    counts = list(freq_map.values())
    return max(counts) - min(counts)

def day14_step2(pairs: Dict[str, int], insertion_rules: Dict[str, Tuple[str, str]]) -> Dict[str, int]:
    next_pairs = defaultdict(int)
    for pair, count in pairs.items():
        for replacement in insertion_rules.get(pair, (pair,)):
            next_pairs[replacement] += count
    return next_pairs


def day14_2(data: Tuple[str, Dict[str, str]]) -> int:
    template = data[0]
    
    pairs = defaultdict(int)
    for i in range(len(template) - 1):
        pairs[template[i:i+2]] += 1

    insertion_rules = {}
    for pair, insertion in data[1].items():
        insertion_rules[pair] = (pair[0] + insertion, insertion + pair[1])

    for i in range(40):
        pairs = day14_step2(pairs, insertion_rules)

    freq_map = defaultdict(int)
    for pair, count in pairs.items():
        freq_map[pair[0]] += count

    # But we need to acocunt of rthe last letter which won't change.
    freq_map[template[-1]] += 1

    counts = list(freq_map.values())
    return max(counts) - min(counts)
    


do(14, 2321, 2399822193707)

[2321, 2399822193707]

# Day 15: Chiton

In [35]:
in15: List[List[int]] =  data(15, parser=lambda l: list(map(int, l)), sep="\n")

In [36]:
# Solvable with A* or Dijkstra's
def manhattan_distance(a: Tuple[int, int], b: Tuple[int, int]) -> int:
    dx = abs(a[0] - b[0])
    dy = abs(a[1] - b[1])
    return dx + dy

def a_star(graph: List[List[int]], start: Tuple[int, int], goal: Tuple[int, int]) -> List[Tuple[int, int]]:
    def get_neighbors(graph: List[List[int]], current: Tuple[int, int]) -> List[Tuple[int, int]]:
        neighbors = []
        for possible in [(-1, 0), (1, 0), (0, 1), (0, -1)]:
            n = (current[0] + possible[0], possible[1] + current[1])
            if n[0] >= 0 and n[0] < len(graph) and n[1] >= 0 and n[1] < len(graph[0]):
                neighbors.append(n)
        return neighbors

    def reconstruct_path(came_from, current):
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        return list(reversed(total_path))

    open_set = []
    came_from = {}
    g_score = {start: 0}
    f_score = {start: manhattan_distance(start, goal)}
    heapq.heappush(open_set, (f_score[start], start))

    while open_set:
        _, current = heapq.heappop(open_set)
        if current == goal:
            return reconstruct_path(came_from, current)
        
        for neighbor in get_neighbors(graph, current):
            tentative_g_score = g_score[current] + graph[neighbor[0]][neighbor[1]]
            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + manhattan_distance(neighbor, goal)
                if neighbor not in open_set:
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))

    return None

def day15_1(risk_map: List[List[int]]) -> int:
    path = a_star(risk_map, (0, 0), (len(risk_map) - 1, len(risk_map[0]) - 1))
    total = 0
    for i in range(1, len(path)):
        r, c = path[i]
        total += risk_map[r][c]
    return total

def tile_risk_map(risk_map: List[List[int]], repeat: int=5) -> List[List[int]]:
    new_risk_map = []
    for r in range(repeat):
        for row in risk_map:
            new_row = []
            for rr in range(repeat):
                for v in row:
                    new_v = v + rr + r
                    if new_v > 9:
                        new_v = (new_v % 9)
                    new_row.append(new_v)
            new_risk_map.append(new_row)
    return new_risk_map
                

def print_day15(risk_map: List[List[int]]) -> None:
    for row in risk_map:
        print(cat(map(str, row)))

def day15_2(risk_map: List[List[int]]) -> int:
    risk_map = tile_risk_map(risk_map, 5)
    path = a_star(risk_map, (0, 0), (len(risk_map) - 1, len(risk_map[0]) - 1))
    total = 0
    for i in range(1, len(path)):
        r, c = path[i]
        total += risk_map[r][c]
    return total

do(15, 523, 2876)

[523, 2876]

# Day 16: Packet Decoder

In [37]:
in16: str = cat([f"{int(f'0x{v}', 16):04b}" for v in data(16)[0]])

In [38]:
@dataclass
class Packet:
    version: int
    type_id: int

    def evaluate(self) -> int:
        raise NotImplementedError()

@dataclass
class LiteralValuePacket(Packet):
    value: int

    def evaluate(self) -> int:
        return self.value

@dataclass
class OperatorPacket(Packet):
    sub_packets: List[Packet]

    def evaluate(self) -> int:
        evaluated_data = [packet.evaluate() for packet in self.sub_packets]
        if self.type_id == 0:
            return sum(evaluated_data)
        if self.type_id == 1:
            return math.prod(evaluated_data)
        if self.type_id == 2:
            return min(evaluated_data)
        if self.type_id == 3:
            return max(evaluated_data)
        if self.type_id == 5:
            assert len(evaluated_data) == 2
            return int(evaluated_data[0] > evaluated_data[1])
        if self.type_id == 6:
            assert len(evaluated_data) == 2
            return int(evaluated_data[0] < evaluated_data[1])
        if self.type_id == 7:
            assert len(evaluated_data) == 2
            return int(evaluated_data[0] == evaluated_data[1])

class PacketParser:
    @staticmethod
    def _parse_number(data: str) -> int:
        return int(data, 2)

    @staticmethod
    def _parse_varint(data: str) -> Tuple[int, int]:
        read_next = True
        i = 0
        bits = []
        while read_next:
            if data[i] == "0":
                read_next = False
            bits.extend(data[i+1:i+5])
            i+=5
        return (PacketParser._parse_number(cat(bits)), i)

    @staticmethod
    def parse_packet(data: str) -> Tuple[Packet, int]:
        version = PacketParser._parse_number(data[0:3])
        type_id = PacketParser._parse_number(data[3:6])
        if type_id == 4:
            value, bits_read = PacketParser._parse_varint(data[6:])
            return (LiteralValuePacket(version=version, type_id=type_id, value=value), 6 + bits_read)
        else:  # Operator Packet
            length_type_id = data[6]
            if length_type_id == "0":
                # 15 bit number follows representing the total length
                total_length = PacketParser._parse_number(data[7:22])
                sub_packets = []
                read_bytes = 0
                start_index = 22
                while read_bytes < total_length:
                    packet, bits_read = PacketParser.parse_packet(data[start_index:])
                    read_bytes += bits_read
                    start_index += bits_read
                    sub_packets.append(packet)
            else:
                # 11 bit number follows representing the number of subpackets
                number_of_subpackets = PacketParser._parse_number(data[7:18])
                start_index = 18
                sub_packets = []
                for i in range(number_of_subpackets):
                    packet, bits_read = PacketParser.parse_packet(data[start_index:])
                    start_index += bits_read
                    sub_packets.append(packet)
            return OperatorPacket(version=version, type_id=type_id, sub_packets=sub_packets), start_index

def day16_1(input: str) -> int:
    packet, bits_read = PacketParser.parse_packet(input)

    def get_version_total(packet):
        total = packet.version
        if isinstance(packet, OperatorPacket):
            total += sum(get_version_total(sub) for sub in packet.sub_packets)
        return total

    return get_version_total(packet)


def day16_2(input: str) -> int:
    packet, bits_read = PacketParser.parse_packet(input)

    return packet.evaluate()

do(16, 934, 912901337844)

[934, 912901337844]

# Day 17: Trick Shot

In [39]:
in17: Tuple[Tuple[int, int], Tuple[int, int]] = tuple([atoms(d[2:], sep="..", ignore=r",") for d in data(17, sep=" ")[2:]])

In [40]:
def day17_simulate_step(pos: Tuple[int, int], velocity: Tuple[int, int]) -> Tuple[Tuple[int, int], Tuple[int, int]]:
    new_pos = (pos[0] + velocity[0], pos[1] + velocity[1])
    new_x_velocity = velocity[0]
    if new_x_velocity > 0:
        new_x_velocity -= 1
    elif new_x_velocity < 0:
        new_x_velocity += 1
    new_velocity = (new_x_velocity, velocity[1] - 1)
    return new_pos, new_velocity

def day17_check_pos_in_target(pos: Tuple[int, int], target: Tuple[Tuple[int, int], Tuple[int, int]]) -> bool:
    return min(target[0]) <= pos[0] <= max(target[0]) and min(target[1]) <= pos[1] <= max(target[1])

def day17_check_pos_past_target(pos: Tuple[int, int], velocity: Tuple[int, int], target: Tuple[Tuple[int, int], Tuple[int, int]]) -> bool:
    return pos[1] < min(target[1]) or (velocity[0] >= 0 and pos[0] > max(target[0])) or (velocity[0] <= 0 and pos[0] < min(target[0]))

def day17_simulate(velocity: Tuple[int, int], target: Tuple[Tuple[int, int], Tuple[int, int]]) -> bool:
    positions = []
    pos = (0, 0)
    vel = velocity
    while not day17_check_pos_past_target(pos, vel, target):
        positions.append(pos)
        if day17_check_pos_in_target(pos, target):
            return True, positions
        pos, vel = day17_simulate_step(pos, vel)
    
    return False, positions

# Tests from the writeup
assert day17_simulate((7, 2), ((20, 30), (-10, -5)))[0]
assert day17_simulate((6, 3), ((20, 30), (-10, -5)))[0]
assert day17_simulate((9, 0), ((20, 30), (-10, -5)))[0]
assert not day17_simulate((17, -4), ((20, 30), (-10, -5)))[0]

# Brute force, not great.
def day17_1(target: Tuple[Tuple[int, int], Tuple[int, int]]) -> int:
    max_y = 0
    vel = (0, 0)
    for x in range(50):
        for y in range(500):
            hit, positions = day17_simulate((x, y), target)
            if hit:
                max_y_pos = max(v for _, v in positions)
                if max_y_pos > max_y:
                    vel = (x, y)
                    max_y = max_y_pos
                
    return max_y

# Brute force, not great.
def day17_2(target: Tuple[Tuple[int, int], Tuple[int, int]]) -> int:
    vels = []
    for x in range(max(target[0]) + 1):
        for y in range(min(target[1]), 1000):
            hit, _ = day17_simulate((x, y), target)
            if hit:
                vels.append((x, y))
                
    return len(vels)

do(17, 13203, 5644)

[13203, 5644]

# Day 18: Snailfish

In [41]:
in18: List[str] = data(18)

In [42]:
@dataclass
class SnailfishLeaf:
    value: int

    def __str__(self) -> str:
        return str(self.value)

    def __eq__(self, other) -> bool:
        return self.value == other.value

@dataclass
class SnailfishNumber:
    left: Union[SnailfishLeaf, "SnailfishNumber"]
    right: Union[SnailfishLeaf, "SnailfishNumber"]
    parent: Optional["SnailfishNumber"]

    def get_root(self) -> "SnailfishNumber":
        if self.parent is None:
            return self
        return self.parent.get_root()

    def magnitude(self) -> int:
        if isinstance(self.left, SnailfishLeaf):
            left = self.left.value
        else:
            left = self.left.magnitude()

        if isinstance(self.right, SnailfishLeaf):
            right = self.right.value
        else:
            right = self.right.magnitude()

        return (3 * left) + (2 * right)

    def __eq__(self, other) -> bool:
        return type(self) == type(other) and self.left == other.left and self.right == other.right

    def add_to_right_neighbor(self, v: int) -> None:
        # Find first node where we are the left child. If we reach None, then there is no right neighbor.
        current = self
        while not current._is_left_child():
            current = current.parent
            # There is no right neighbor
            if current is None:
                return
        current = current.parent
        right_subtree = current.right
        # Add v to the leftmost child of right_subtree
        while type(right_subtree) == SnailfishNumber:
            right_subtree = right_subtree.left

        assert type(right_subtree) == SnailfishLeaf

        right_subtree.value += v

    def add_to_left_neighbor(self, v: int) -> None:
        # Find first node where we are the right child. If we reach None, then there is no left neighbor.
        current = self
        while not current._is_right_child():
            current = current.parent
            # There is no left neighbor
            if current is None:
                return
        current = current.parent
        left_subtree = current.left
        # Add v to the leftmost child of right_subtree
        while type(left_subtree) == SnailfishNumber:
            left_subtree = left_subtree.right

        assert type(left_subtree) == SnailfishLeaf

        left_subtree.value += v
        
    def _is_left_child(self) -> bool:
        if self.parent is None:
            return False
        return self.parent.left is self

    def _is_right_child(self) -> bool:
        if self.parent is None:
            return False
        return self.parent.right is self
        

    def __add__(self, other: "SnailfishNumber") -> "SnailfishNumber":
        new_root = SnailfishNumber(left=self, right=other, parent=None)
        assert self.parent == None
        assert other.parent == None
        self.parent = new_root
        other.parent = new_root
        return day18_reduce_number(new_root)

    def __str__(self) -> str:
        return f"[{self.left},{self.right}]"

    @staticmethod
    def from_list(l: List, parent: Optional["SnailfishNumber"]=None) -> "SnailfishNumber":
        assert len(l) == 2
        current = SnailfishNumber(parent=parent, left=None, right=None)
        if type(l[0]) == int:
            left = SnailfishLeaf(value=l[0])
        else:
            left = SnailfishNumber.from_list(l[0], parent=current)
        
        if type(l[1]) == int:
            right = SnailfishLeaf(value=l[1])
        else:
            right = SnailfishNumber.from_list(l[1], parent=current)
        current.left = left
        current.right = right
        
        return current

    @staticmethod
    def from_string(s: str) -> "SnailfishNumber":
        return SnailfishNumber.from_list(eval(s))


def day18_explode_number(sn: SnailfishNumber) -> SnailfishNumber:
    assert type(sn.left) == SnailfishLeaf and type(sn.right) == SnailfishLeaf
    left = sn.left.value
    right = sn.right.value
    sn.add_to_right_neighbor(right)
    sn.add_to_left_neighbor(left)
    assert sn.parent is not None
    # Remove the node from the tree
    parent = sn.parent
    sn.parent = None
    if parent.left is sn:
        parent.left = SnailfishLeaf(value=0)
    else:
        parent.right = SnailfishLeaf(value=0)

    return parent.get_root()

def day18_split_number(sn: SnailfishNumber) -> SnailfishNumber:
    if type(sn.left) == SnailfishLeaf and sn.left.value >= 10:
        v = sn.left.value/2
        left = SnailfishLeaf(value=math.floor(v))
        right = SnailfishLeaf(value=math.ceil(v))
        node = SnailfishNumber(parent=sn, left=left, right=right)
        sn.left = node
    elif type(sn.right) == SnailfishLeaf and sn.right.value >= 10:
        v = sn.right.value/2
        left = SnailfishLeaf(value=math.floor(v))
        right = SnailfishLeaf(value=math.ceil(v))
        node = SnailfishNumber(parent=sn, left=left, right=right)
        sn.right = node

    return sn.get_root()

def day18_try_explode(sn: SnailfishNumber) -> Tuple[bool, SnailfishNumber]:
    depth = 0
    def dfs(sn: SnailfishNumber, depth: int) -> Tuple[bool, SnailfishNumber]:
        if depth >= 4:
            return True, day18_explode_number(sn)

        if isinstance(sn.left, SnailfishNumber):
            reduced, new_number = dfs(sn.left, depth + 1)
            if reduced:
                return reduced, new_number

        if isinstance(sn.right, SnailfishNumber):
            reduced, new_number = dfs(sn.right, depth + 1)
            if reduced:
                return reduced, new_number
            
        return False, sn

    return dfs(sn, depth)

def day18_try_split(sn: SnailfishNumber) -> Tuple[bool, SnailfishNumber]:
    def dfs(sn: SnailfishNumber) -> Tuple[bool, SnailfishNumber]:
        if isinstance(sn.left, SnailfishLeaf):
            if sn.left.value >= 10:
                return True, day18_split_number(sn)

        if isinstance(sn.left, SnailfishNumber):
            reduced, new_number = dfs(sn.left)
            if reduced:
                return reduced, new_number

        if isinstance(sn.right, SnailfishLeaf):
            if sn.right.value >= 10:
                return True, day18_split_number(sn)

        if isinstance(sn.right, SnailfishNumber):
            reduced, new_number = dfs(sn.right)
            if reduced:
                return reduced, new_number
            
        return False, sn

    return dfs(sn)

def day18_reduce_number_step(sn: SnailfishNumber) -> Tuple[bool, SnailfishNumber]:
    depth = 0
    def dfs(sn: SnailfishNumber, depth: int) -> Tuple[bool, SnailfishNumber]:
        if depth >= 4:
            return True, day18_explode_number(sn)

        if isinstance(sn.left, SnailfishLeaf):
            if sn.left.value >= 10:
                return True, day18_split_number(sn)

        if isinstance(sn.left, SnailfishNumber):
            reduced, new_number = dfs(sn.left, depth + 1)
            if reduced:
                return reduced, new_number

        if isinstance(sn.right, SnailfishLeaf):
            if sn.right.value >= 10:
                return True, day18_split_number(sn)

        if isinstance(sn.right, SnailfishNumber):
            reduced, new_number = dfs(sn.right, depth + 1)
            if reduced:
                return reduced, new_number
            
        return False, sn

    return dfs(sn, depth)

def day18_reduce_number(sn: SnailfishNumber) -> SnailfishNumber:
    num = sn
    while True:
        did_explode, num = day18_try_explode(num)
        if did_explode:
            continue
        did_split, num = day18_try_split(num)
        if did_split:
            continue
        break        
    return num

def day18_sum_list(sns: List[SnailfishNumber]) -> SnailfishNumber:
    current = first(sns)
    for sn in rest(sns):
        current = current + sn
    return current

def day18_1(unparsed_sns: List[str]) -> int:
    sns = [SnailfishNumber.from_string(sn) for sn in unparsed_sns]
    return day18_sum_list(sns).magnitude()

def day18_2(unparsed_sns: List[str]) -> int:
    sns = [SnailfishNumber.from_string(sn) for sn in unparsed_sns]
    possible_pairs = combinations(unparsed_sns, 2)
    return max([
        max([
            (SnailfishNumber.from_string(a) + SnailfishNumber.from_string(b)).magnitude(), 
            (SnailfishNumber.from_string(b) + SnailfishNumber.from_string(a)).magnitude()])
            for a, b in possible_pairs])

def check_reduce_step(start: str, want: str) -> None:
    got = day18_reduce_number_step(SnailfishNumber.from_string(start))[1]
    assert got == SnailfishNumber.from_string(want), f"Reducing: {start} - got: {got}, expected: {want}"

def check_add(a: str, b: str, want: str) -> None:
    got = SnailfishNumber.from_string(a) + SnailfishNumber.from_string(b)
    assert got == SnailfishNumber.from_string(want), f"Adding: {a} + {b} - got: {got}, expected: {want}"

def check_magnitude(a: str, want: int) -> None:
    got = SnailfishNumber.from_string(a).magnitude()
    assert got == want, f"Magnitude: {a} - got: {got}, expected: {want}"

def check_sum_list(l: List[str], want: str) -> None:
    got = day18_sum_list(mapt(SnailfishNumber.from_string, l))
    assert got == SnailfishNumber.from_string(want), f"Summing List: {l} - got: {got}, expected: {want}"


check_reduce_step("[[[[[9,8],1],2],3],4]", "[[[[0,9],2],3],4]")
check_reduce_step("[7,[6,[5,[4,[3,2]]]]]", "[7,[6,[5,[7,0]]]]")
check_reduce_step("[[6,[5,[4,[3,2]]]],1]", "[[6,[5,[7,0]]],3]")
check_reduce_step("[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]", "[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]")
check_reduce_step("[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]", "[[3,[2,[8,0]]],[9,[5,[7,0]]]]")
check_add("[[[[4,3],4],4],[7,[[8,4],9]]]", "[1,1]", "[[[[0,7],4],[[7,8],[6,0]]],[8,1]]")
check_add("[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]", "[[[5,[2,8]],4],[5,[[9,9],0]]]", "[[[[7,0],[7,8]],[[7,9],[0,6]]],[[[7,0],[6,6]],[[7,7],[0,9]]]]")
check_add("[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]", "[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]", "[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]")
check_magnitude("[[1,2],[[3,4],5]]", 143)
check_magnitude("[[[[0,7],4],[[7,8],[6,0]]],[8,1]]", 1384)
check_magnitude("[[[[1,1],[2,2]],[3,3]],[4,4]]", 445)
check_magnitude("[[[[3,0],[5,3]],[4,4]],[5,5]]", 791)
check_magnitude("[[[[5,0],[7,4]],[5,5]],[6,6]]", 1137)
check_magnitude("[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]", 3488)
check_sum_list(["[1,1]", "[2,2]", "[3,3]", "[4,4]"], "[[[[1,1],[2,2]],[3,3]],[4,4]]")
check_sum_list(["[1,1]", "[2,2]", "[3,3]", "[4,4]", "[5,5]"], "[[[[3,0],[5,3]],[4,4]],[5,5]]")
check_sum_list(["[1,1]", "[2,2]", "[3,3]", "[4,4]", "[5,5]", "[6,6]"], "[[[[5,0],[7,4]],[5,5]],[6,6]]")

do(18, 2541, 4647)

[2541, 4647]

# Day 19: Beacon Scanner

In [43]:
Vec3 = Tuple[int, int, int]
Scanner = List[Vec3]

in19: List[Scanner] = [[atoms(beacon, sep=",") for beacon in rest(scanner.split("\n"))] for scanner in data(19, sep="\n\n")]

In [44]:
def day19_distance(p1: Vec3, p2: Vec3) -> int:
    x1, y1, z1 = p1
    x2, y2, z2 = p2
    return abs(x1 - x2) + abs(y1 - y2) + abs(z1 - z2)

def square_distance(a: Vec3, b: Vec3) -> int:
    return sum((xa - xb) ** 2 for xa, xb in zip(a, b))

def rotate_x(theta):
    return np.array([[1, 0, 0],
                     [0, math.cos(theta),-math.sin(theta)],
                     [0, math.sin(theta), math.cos(theta)]])
  
def rotate_y(theta):
    return np.array([[math.cos(theta), 0, math.sin(theta)],
                      [0, 1, 0],
                      [-math.sin(theta), 0, math.cos(theta)]])
  
def rotate_z(theta):
    return np.array([[math.cos(theta), -math.sin(theta), 0],
                     [math.sin(theta),  math.cos(theta), 0],
                     [0,0 ,1]])

def rotate(psi, theta, phi):
     return rotate_z(psi) @ rotate_y(theta) @ rotate_z(phi)

def get_all_rotations():
    rotations = []
    for i in range(4):
        for j in range(4):
            for k in range(4):
                phi = i * np.pi/2
                theta = j * np.pi/2
                psi = k * np.pi/2
                rot = np.array(np.round(rotate(psi, theta, phi), decimals=2), dtype=int)
                unique = True
                for rs in rotations:
                    if (rs == rot).all():
                        unique = False
                if unique:
                    rotations.append(rot)
    return rotations

# All 24 rotations generated
all_rotations = get_all_rotations()

def try_transform_scanner(a: Scanner, b: Scanner) -> Tuple[Scanner, Vec3]:
    """Try to find common beacons and merge b into a."""
    a_distances = {square_distance(x, y): (x, y) for (x, y) in combinations(a, 2)}
    b_distances = {square_distance(x, y): (x, y) for (x, y) in combinations(b, 2)}

    common_distances = set(a_distances.keys()) & set(b_distances.keys())
    if len(common_distances) < (12 * 11)/2:
        return [], (0, 0, 0)
    
    for c in common_distances:
        target_point = np.array(a_distances[c][0])
        start_point = np.array(b_distances[c][0])
        for rotation in all_rotations:
            rotated_point = rotation @ start_point
            translation = target_point - rotated_point

            transformed_points = set([tuple((rotation @ np.array(v)) + translation) for v in b])
            target_points = set(a)
            if len(transformed_points & target_points) >= 12:
                return list(transformed_points), tuple(translation)
    
    return [], (0, 0, 0)
    

def find_all_beacons_and_scanners(scanners: List[Scanner]) -> Tuple[Set[Vec3], List[Vec3]]:
    # Known beacons in scanner0's frame.
    transformed_scanners: Dict[int, Scanner] = {0: scanners[0]}

    # Locations of the scanners scanner 0's frame.
    scanner_locations: List[Vec3] = [(0, 0, 0)] * len(scanners)

    # Scanner indices that are already converted to the 0 scanner's base frame.
    analyzed_scanners: Set[int] = set([0])

    while len(analyzed_scanners) < len(scanners):
        found = False
        for analyzed_scanner_index in list(analyzed_scanners):
            if found:
                break
            target = transformed_scanners[analyzed_scanner_index]
            for i, s in enumerate(scanners):
                if i not in analyzed_scanners:
                    new_scanner, translation = try_transform_scanner(target, s)
                    if new_scanner:
                        analyzed_scanners.add(i)
                        transformed_scanners[i] = new_scanner
                        scanner_locations[i] = translation
                        found = True
                        break
    
    return set().union(*mapt(set, transformed_scanners.values())), scanner_locations
    
def day19_1(scanners: List[Scanner]) -> int:
    beacons, locations = find_all_beacons_and_scanners(scanners)
    return len(beacons)

def day19_2(scanners: List[Scanner]) -> int:
    beacons, locations = find_all_beacons_and_scanners(scanners)
    return max(day19_distance(x, y) for (x, y) in combinations(locations, 2))
    

do(19, 414, 13000)


[414, 13000]

# Day 20: Trench Map

In [45]:

Image = Sequence[Sequence[bool]]
Algorithm = Sequence[bool]

def day20_parse(data: Tuple[str, str]) -> Tuple[List[bool], Image]:
    algo = mapt(lambda v: v=="#", data[0])

    image = [[v == "#" for v in line] for line in data[1].split("\n")]
    return algo, image

in20: Tuple[Algorithm, Image] = day20_parse(data(20, sep="\n\n"))

In [46]:
InfiniteImage = Tuple[Image, bool]

def day20_print_image(image: Image) -> None:
    for row in image:
        for v in row:
            c = "#" if v else "."
            print(c, end="")
        print("")

def pad_image(image: Image, padding: bool) -> Image:
    """Pad an image with 2 of the padding in every direction."""
    new_rows = len(image) + 4
    new_cols = len(image[0]) + 4
    new_image = []
    for i in range(new_rows):
        old_row = i - 2
        if old_row < 0 or old_row >= len(image):
            new_image.append([padding] * new_cols)
        else:
            new_row = ([padding] * 2) + image[old_row] + ([padding] * 2)
            new_image.append(new_row)
    return new_image

def day20_enhance(inf_image: InfiniteImage, algorithm: Algorithm) -> InfiniteImage:
    padded_image = pad_image(inf_image[0], inf_image[1])
    new_image = []
    for i in range(1, len(padded_image) - 1):
        new_row = []
        for j in range(1, len(padded_image[i]) - 1):
            binary_bools = [padded_image[i + r][j + c] for (r, c) in [
                (-1, -1), (-1, 0), (-1, 1),
                (0, -1), (0, 0), (0, 1),
                (1, -1), (1, 0), (1, 1),
            ]]
            algo_idx = int("".join(["1" if v else "0" for v in binary_bools]), 2)
            algo_v = algorithm[algo_idx]
            new_row.append(algo_v)
        new_image.append(new_row)

    if inf_image[1]:
        next_inf = algorithm[511]
    else:
        next_inf = algorithm[0]

    return new_image, next_inf

def day20_1(data: Tuple[Algorithm, Image]) -> int:
    algorithm, image = data
    assert len(algorithm) == 512
    infinite_image = (image, False)
    for i in range(2):
        infinite_image = day20_enhance(infinite_image, algorithm)

    assert not infinite_image[1]
    return sum([quantify(row) for row in infinite_image[0]])

def day20_2(data: Tuple[Algorithm, Image]) -> int:
    algorithm, image = data
    assert len(algorithm) == 512
    infinite_image = (image, False)
    for i in range(50):
        infinite_image = day20_enhance(infinite_image, algorithm)

    assert not infinite_image[1]
    return sum([quantify(row) for row in infinite_image[0]])




do(20, 5479, 19012)

[5479, 19012]

# Day 21: Dirac Dice

In [47]:
in21: Tuple[int, int] = mapt(lambda v: v[-1], data(21, parser=atoms))

In [48]:
class DeterministicDie:
    def __init__(self, max: int=100):
        self._next_value = 1
        self._max = max
        self._count = 0
    
    def roll(self) -> int:
        v = self._next_value
        self._next_value = (self._next_value % self._max) + 1
        self._count += 1
        return v

    def get_roll_count(self) -> int:
        return self._count

GameState = Tuple[Tuple[int, int], Tuple[int, int]]        

def day21_player_turn(current_position: int, die: DeterministicDie) -> int:
    roll_result = sum(die.roll() for i in range(3))
    return day21_get_next_position(current_position, roll_result)

def day21_get_next_position(current_position: int, roll_result: int) -> int:
    return ((current_position + roll_result - 1 ) % 10) + 1

def day21_play_game(starting_positions: Tuple[int, int], die: DeterministicDie) -> GameState:
    scores = [0, 0]
    positions = list(starting_positions)

    while True:
        for player in range(len(scores)):
            next_position = day21_player_turn(positions[player], die)
            scores[player] += next_position
            positions[player] = next_position
            if scores[player] >= 1000:
                return tuple(positions), tuple(scores)

def day21_1(starting_positions: Tuple[int, int]) -> int:
    d = DeterministicDie()
    positions, scores = day21_play_game(starting_positions, d)

    return min(scores) * d.get_roll_count()

def day21_2(starting_positions: Tuple[int, int]) -> int:
    # Mapping from roll to number of ways that can happen.
    dirac_dice_rolls = {3: 1, 4: 3, 5: 6, 6: 7, 7: 6, 8: 3, 9: 1}
    games = deque([]) # Deque[Tuple[GameState, int, int]], which player's turn it is and number of universes.
    games.append(((starting_positions, (0, 0)), 0, 1))
    won_games = [0, 0]
    while games:
        game = games.popleft()
        game_state = game[0]
        player = game[1]
        num_universes = game[2]
        for roll, frequency in dirac_dice_rolls.items():
            next_position = day21_get_next_position(game_state[0][player], roll)
            next_score = next_position + game_state[1][player]
            new_universe_count = frequency * num_universes
            if next_score >= 21:
                won_games[player] += new_universe_count
            else:
                position = list(game_state[0])
                position[player] = next_position
                score = list(game_state[1])
                score[player] = next_score
                games.append(((tuple(position), tuple(score)), 1 if player == 0 else 0, new_universe_count))

    return max(won_games)


do(21, 412344, 214924284932572)

[412344, 214924284932572]

# Day 22: Reactor Reboot

In [49]:
Range = Tuple[int, int]
Cube = Tuple[Range, Range, Range]
RebootStep = Tuple[bool, Cube]

def day22_parse(line: str) -> RebootStep:
    action, ranges_str = line.split(' ')
    on = action == "on"
    ranges = ranges_str.split(",")
    parsed_ranges = tuple([mapt(int, range_str[2:].split("..")) for range_str in ranges])
    return on, parsed_ranges

in22: List[RebootStep] = data(22, parser=day22_parse)

In [50]:
def day22_1(reboot_steps: List[RebootStep]) -> int:
    active_cubes = set()
    for step in reboot_steps:
        on = step[0]
        ranges = step[1]
        if all(all(-50 <= v <= 50 for v in r) for r in ranges):
            current_set = set()
            for x in range(ranges[0][0], ranges[0][1] + 1):
                for y in range(ranges[1][0], ranges[1][1] + 1):
                    for z in range(ranges[2][0], ranges[2][1] + 1):
                        current_set.add((x, y, z))
            if on:
                active_cubes |= current_set
            else:
                active_cubes -= current_set
    return len(active_cubes)

def day22_intersect(from_step: RebootStep, into: RebootStep) -> RebootStep:
    # Max the mins and Min the maxes
    new_cube = tuple(((max(a[0], b[0]), min(a[1], b[1])) for a, b in zip(from_step[1], into[1])))
    new_step = (not into[0], new_cube)
    return None if any(d[0] > d[1] for d in new_cube) else new_step


def day22_2(reboot_steps: List[RebootStep]) -> int:
    merged_steps = []
    for step in reboot_steps:
        to_add = []
        for m_s in merged_steps:
            intersection = day22_intersect(step, m_s)
            if intersection is not None:
                to_add.append(intersection)
        if step[0]:
            to_add.append(step)
        merged_steps.extend(to_add)

    count = 0
    for step in merged_steps:
        ranges = step[1]
        cube_size = math.prod(((r[1] - r[0]) + 1) for r in ranges)
        sign = 1 if step[0] else -1
        count += sign * cube_size

    return count


do(22, 596989, 1160011199157381)

[596989, 1160011199157381]